In [2]:
import cv2
import numpy as np
import copy

In [3]:
cv2.__version__

'3.4.0'

# actor from given actor list will be listed as desired object

In [8]:
actor_array = ['SM_DenkMitEdelstahlReinigerSpray_18','SM_CalgonitFinishKlarspueler_3','SM_CalgonitFinishVorratspack_12']
actor_array = np.array(actor_array)

In [4]:
# dirName = 'F:/unreal_cv_documentation/detect_ROI_for_multi_actor/new_image/'

dirName = '/media/atif/0820209220208930/unreal_cv_documentation/detect_ROI_for_multi_actor/new_image/'
image_type = '.png'

In [5]:
rgb_image = cv2.imread(str(dirName)+'lit_new_2019-06-26_22-41-31'+str(image_type))
mask_image = cv2.imread(str(dirName)+'mask_new_2019-06-26_22-41-31'+str(image_type))

# from the other code named detect_roi_while_every_actor_has_different_color_in_mask.ipynb we have already saved the image and a text file where all RGB info of desired object has stored in a text file
# from the text file those RGB info will be fetched and store in an array to use later

In [6]:
import numpy as np
# store=[]
store = list()
with open(str(dirName)+'color_info2019-06-26_22-41-31.txt') as f1:
    a=f1.readlines()
    for i in a:
        k=i.split('\n')
        m=k[0].split(' ')
        store.append(m)
store=np.array(store,dtype=int)
r,g,b,a=[store[:,i] for i in range(len(store[0]))]

In [9]:
for in_actor,i in enumerate(actor_array):
    print(r[in_actor])
    print(g[in_actor])
    print(b[in_actor])
    print('\n')

0
127
127


0
127
255


255
127
0




In [10]:
img = np.zeros((mask_image.shape[0],mask_image.shape[1],mask_image.shape[2]), np.uint8) #create a black coloured image
test_copy_img = copy.copy(img) #here all object with white color will be passed
test_copy_img_color = copy.copy(img) #here all object with their mask color will be passed

for in_actor,i in enumerate(actor_array):
    for rows in range (0, mask_image.shape[0]):
        for cols in range(0, mask_image.shape[1]):
            if mask_image[rows][cols][0]==b[in_actor] and mask_image[rows][cols][1]==g[in_actor] and mask_image[rows][cols][2]==r[in_actor]:
                test_copy_img[rows][cols] = 255 #for white color 255
                test_copy_img_color[rows][cols]=mask_image[rows][cols]
                
test_copy_image_color_mask = copy.copy(test_copy_img_color)
                
imgray=cv2.cvtColor(test_copy_img,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,127,255,0)
image, contours, hierarchy =  cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

roi_list = []
rgb_image_copy = copy.copy(rgb_image)
for t in range (0,len(actor_array),1):
    if len(contours)>0:
        print('length is: ',len(contours))
        cnt = contours[t]
        x,y,w,h = cv2.boundingRect(cnt)
        roi_rgb_image=cv2.rectangle(rgb_image_copy,(x,y),(x+w,y+h),(0,255,0),1)
        crop_img = roi_rgb_image[y:y+h, x:x+w]
        roi_black_mask = cv2.rectangle(test_copy_image_color_mask,(x,y),(x+w,y+h),(0,255,0),1)
        print('info of ',t,' contour ',x,',',y,',',x+w,',',y+h)
        x=x
        y=y
        x_1=x+w
        y_1=y+h
        
        roi_point = [x,y,x_1,y_1]
        roi_list.append(roi_point)

length is:  3
info of  0  contour  375 , 198 , 416 , 270
length is:  3
info of  1  contour  472 , 194 , 534 , 263
length is:  3
info of  2  contour  271 , 189 , 298 , 273


# store the ROI in a text file to use later

In [7]:
f = open(str(dirName)+'roi_info.txt', 'a')
f = open(str(dirName)+'roi_info.txt', 'r+')
f.truncate(0)

for i in roi_list:
    ww = str(i)
    ww=ww.replace('[','')
    ww=ww.replace(']','')
    ww=ww.replace(',','')
    f.write(ww)
    f.write("\n")
f.close()

# do RGB image black color except desired actor

In [11]:
a_copy = copy.copy(test_copy_img)
rgb_black = copy.copy(rgb_image)

for rows in range (0, a_copy.shape[0]):
    for cols in range (0, a_copy.shape[1]):
        if a_copy[rows][cols][0]==0 and a_copy[rows][cols][1]==0 and a_copy[rows][cols][2]==0:
            rgb_black[rows][cols]=0
        else:
            pass

# draw ROI of desired object in the RGB_black image

In [12]:
rgb_black_roi = copy.copy(rgb_black)

for i in roi_list:
    x_new = i[0]
    y_new = i[1]
    x_w_new = i[2]
    y_h_new = i[3]
    
    roi_black_rgb = cv2.rectangle(rgb_black_roi,(x_new,y_new),(x_w_new,y_h_new),(0,255,0),1)

In [14]:
cv2.imshow('rgb', rgb_image)
cv2.imshow('mask', mask_image)
cv2.imshow('test_copy_img',test_copy_img)
cv2.imshow('test_copy_img_color', test_copy_img_color)
cv2.imshow('roi_rgb_image', roi_rgb_image)
cv2.imshow('roi_black_mask',test_copy_image_color_mask)
cv2.imshow('rgb_black', rgb_black)
cv2.imshow('roi_black_rgb', roi_black_rgb)
cv2.imshow('crop_img',crop_img)


if cv2.waitKey() == ord('q'): #press q to close the output image window
        cv2.destroyAllWindows()

In [11]:
cv2.imwrite(str(dirName)+'crop_image'+str(image_type), crop_img)

True